In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 7it [00:32,  3.27s/it]                       

In [2]:
import variables
import contraintes
import importlib
import utils
importlib.reload(variables) #to remive in final .py
importlib.reload(contraintes) #to remive in final .py
importlib.reload(utils)

<module 'utils' from '/Users/elliot09/SNCF-Ressources-Optimisation/utils.py'>

In [3]:
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            row += 1

    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m)


In [4]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
B, OCCUPATIONS = contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
final_train_task = var_dict["essai de frein départ"][final_train_task]
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)

m.update()
m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")

show_solution(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 431.84it/s]


KeyError: 'Machine'

Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [ ]:
##### Respect du nombre de voies de chantier #####
#debug
# print(B)



taches_df = dataframes["taches_df"]
chantiers_df = dataframes["chantiers_df"]
chantier_cycles = {}
for chantier in set(taches_df["Chantier"].values):
    chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
                                                                                        taches_df[taches_df["Chantier"] == chantier].Ordre.max()]["Type de tache humaine"].iloc[0]
    chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
                                                                                    taches_df[taches_df["Chantier"] == chantier].Ordre.min()]["Type de tache humaine"].iloc[0]
print(chantier_cycles)
def get_all_tasks_by_name(name):
    # print("get_all : ", name)
    return var_dict[name].keys()
def b(tache_i, tache_j, tache_name, duree=0):
    if(tache_name,tache_j,tache_i) in B.keys():
        # print("HEY === ",(tache_name,tache_j,tache_i))
        bb = B[tache_name,tache_j,tache_i]
        # print(bb.X, end=",")
        return bb.X
    elif (tache_name,tache_i,tache_j) in B.keys():
        # print("HEY ELIF === ",(tache_name,tache_i,tache_j))
        bb = B[tache_name,tache_i,tache_j]
        # print(1-bb.X, end=",")
        return 1-bb.X
            
        
    else:
        print("ERROR === ", (tache_name,tache_i,tache_j))
               

OCCUPATIONS = []
def add_occupation_constr(chantier, tache_debut):
    
    occupation = 0 
    occupation = sum([b(tache_chantier,
                                tache_debut,
                                chantier_cycles[chantier, "start"])
                            for tache_chantier in get_all_tasks_by_name(chantier_cycles[chantier, "start"])])
    duree = taches_df[taches_df["Type de tache humaine"]
                        == chantier_cycles[chantier, "end"]]["Durée"].iloc[0]
    # print(occupation)
    # print("-----------")
    
    temp = sum([b(tache_chantier,
                                            tache_debut,
                                            chantier_cycles[chantier, "end"],
                                            duree= duree)
                                        for tache_chantier in get_all_tasks_by_name(chantier_cycles[chantier, "end"])])
    # print(temp)
    occupation = occupation -temp
    if occupation!= 0 :print("NON ZERO",occupation)
    OCCUPATIONS.append(occupation)
# for chantier in set(chantiers_df["Chantier"].values):
for chantier in ["WPY_FOR"]:
    print(f"Adding occupation constrainst for: {chantier}...")
    print("start task: ",chantier_cycles[chantier, "start"])
    print("end task: ",chantier_cycles[chantier, "end"])
    for tache_debut in get_all_tasks_by_name(chantier_cycles[chantier, "start"]):
        add_occupation_constr(chantier, tache_debut)
print(OCCUPATIONS)
# print(B[("arrivée Reception","43124608/08/202200:00:0011:0408/08/2022","45023708/08/202200:00:0011:4708/08/2022")])

{('WPY_REC', 'end'): 'débranchement', ('WPY_REC', 'start'): 'arrivée Reception', ('WPY_FOR', 'end'): 'dégarage / bouger de rame', ('WPY_FOR', 'start'): 'appui voie + mise en place câle', ('WPY_DEP', 'end'): 'essai de frein départ', ('WPY_DEP', 'start'): 'essai de frein départ'}
Adding occupation constrainst for: WPY_FOR...
start task:  appui voie + mise en place câle
end task:  dégarage / bouger de rame
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO 1.0
NON ZERO

<ipython-input-10-e1c2cf89921c>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
<ipython-input-10-e1c2cf89921c>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
<ipython-input-10-e1c2cf89921c>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
<ipython-input-10-e1c2cf89921c>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
<ipython-input-10-e1c2cf89921c>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end

In [ ]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 428.66it/s]
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:175: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:175: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:175: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:175: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:175: UserWarning: Boolean Series key will be reindexed to

debug sillons
5408609/08/202203:5023:5914/08/2022
[2347]
43124608/08/202200:0011:0408/08/2022
debug sillons
5408611/08/202203:5023:5914/08/2022
[159]
45023708/08/202200:0011:4710/08/2022

/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:183: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = co


debug sillons
5408612/08/202203:5023:5914/08/2022
[160, 225, 323, 687, 1063, 1106, 1497, 1703, 2349]
45023708/08/202200:0011:4711/08/2022
5673008/08/202200:0002:3211/08/2022
5673008/08/202200:0002:3211/08/2022
4126408/08/202200:0022:3310/08/2022
5423408/08/202200:0022:0110/08/2022
4126408/08/202200:0022:3310/08/2022
4126408/08/202200:0022:3310/08/2022
43164008/08/202200:0011:0911/08/2022
43124608/08/202200:0011:0411/08/2022
debug sillons

/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:183: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = co

...

/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:183: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:183: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:190: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_ar

compteur for horaires 217
Adding occupation constrainst for: WPY_DEP...


/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:263: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:265: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:263: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "end"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressources-Optimisation/contraintes.py:265: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressour

Adding occupation constrainst for: WPY_REC...
Adding occupation constrainst for: WPY_FOR...


({('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '40000609/08/202215:4623:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '40001609/08/202222:4623:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '40011009/08/202220:5723:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '43103509/08/202215:0223:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '43109109/08/202206:1923:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '4420309/08/202217:5523:5914/08/2022'): <gurobi.Var *Awaiting Model Update*>,
  ('essai de frein départ',
   '40000609/08/202215:4623:5914/08/2022',
   '4424109/

In [ ]:
show_solution(m)

Objectif minimal:  3889.0
